In [16]:
# https://python.langchain.com/docs/integrations/toolkits/csv
! pip install langchain -q
! pip install openai -q
! pip install clarifai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
fitter 1.6.0 requires tqdm<5.0.0,>=4.65.1, but you have tqdm 4.64.1 which is incompatible.
jupyterlab-lsp 4.2.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.3.1 requires scipy<1.11,>=1.4.1, but you have scipy 1.11.2 which is incompatible.


In [12]:
import pandas as pd
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.agents import create_csv_agent

In [17]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("clarifai")
secret_value_1 = user_secrets.get_secret("openai")

In [13]:
agent = create_csv_agent(
    ChatOpenAI(temperature=0, openai_api_key=secret_value_1),
    "https://github.com/bsenst/streamlit-llm/raw/main/notebooks/csv/devices.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [15]:
# agent.run("how many unique devices does the data contain?")

In [18]:
# https://python.langchain.com/docs/integrations/llms/clarifai
# Import the required modules
from langchain.llms import Clarifai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [54]:
# USER_ID = "openai"
# APP_ID = "chat-completion"
# MODEL_ID = "GPT-3_5-turbo"

USER_ID = "meta"
APP_ID = "Llama-2"
MODEL_ID = "llama2-13b-chat"

# You can provide a specific model version as the model_version_id arg.
# MODEL_VERSION_ID = "MODEL_VERSION_ID"

In [55]:
template = """{question}
{context}"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [56]:
# Initialize a Clarifai LLM
clarifai_llm = Clarifai(
    pat=secret_value_0, user_id=USER_ID, app_id=APP_ID, model_id=MODEL_ID
)

In [57]:
llm_chain = LLMChain(prompt=prompt, llm=clarifai_llm)

In [58]:
df = pd.read_csv("https://github.com/bsenst/streamlit-llm/raw/main/notebooks/csv/devices.csv")

In [59]:
df.PATIENT.value_counts()

PATIENT
85a0da4b-6e8d-492c-976b-748e39daf4ef    2
fd1e9a6d-5b50-4a6f-bd61-c884ab89f5c6    2
668ad49e-46d7-45ec-a4b4-dab0961d180b    2
2c71dd97-7085-416a-aa07-d675bbe3adf2    2
cb1be644-93a5-4208-8776-f5feece6967a    1
                                       ..
8507a0b0-b667-4a07-90f1-8cfb2193dcd8    1
16fe231f-5db5-4213-9ce0-ba1944eac066    1
2be232c2-45cd-4621-b16a-9bd0ff62181a    1
afb2122f-e8e3-4ae4-9911-b8e2dc4f567d    1
023a7d29-32b3-4db5-89c8-b88bd7582ec0    1
Name: count, Length: 74, dtype: int64

In [60]:
df.DESCRIPTION.value_counts()

DESCRIPTION
Implantable defibrillator  device (physical object)    47
Coronary artery stent (physical object)                21
Implantable cardiac pacemaker (physical object)        10
Name: count, dtype: int64

# Understanding medical devices with LLM

In [61]:
question = "Which organ system is related to the following medical devices?"
context = " ".join(df.DESCRIPTION.unique())

# OpenAI's model maximum context length is 4097 tokens.
llm_chain.run({'question':question, 'context':context})

"\n\nAnswer: Cardiovascular system.\n\nExplanation: The medical devices mentioned are all related to the cardiovascular system. An implantable defibrillator is a device that is implanted in the chest to monitor and regulate the heart's rhythm. A coronary artery stent is a device that is inserted into the coronary arteries to improve blood flow to the heart. An implantable cardiac pacemaker is a device that is implanted in the chest to regulate the heart's rhythm. All of these devices are used to treat various cardiovascular conditions and are related to the cardiovascular system."